For eNE recognition we need a map of a form {PMID:publishing year}, with an entry for every PubMed paper.



In [ ]:
#mount google drive to use data
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import re
import json

def create_paper_year_map(pubmed_results_file):
  """
  create a dictionary that contains mapping of PMIDs of PubMed 
  articles to its publishing year
  """
  paper_year_map = {}
  year_regex = r"[0-9]{4,4} [A-Z,a-z]{3,3} [0-9]{1,2};"
  short_year_regex =r"[0-9]{4,4} [A-Z,a-z]{3,3};"
  exception_year_regex = r"[0-9]{4,4} [A-Z,a-z]{3,3}-"
  epub_regex = r"Epub [0-9]{4,4} "
  pmid_regex = r"PMID:[ ]{0,1}[0-9]+"
  
  with open(pubmed_results_file, encoding="utf8") as f:    
    current_text = ""   
    for line in f:  
      clean_line = line.replace("\n", " ")     
      current_text += clean_line      
      if clean_line == " ":  
         print("---") 
         pmid_match = re.search(pmid_regex, current_text)  
         if pmid_match: 
           pmid_clean = pmid_match.group().replace("PMID: ", "").replace(";", "")
           epub_match = re.search(epub_regex, current_text)
           if epub_match:
             clean_epub = epub_match.group().split(" ")[1]
             paper_year_map[pmid_clean] = clean_epub
             current_text = "" 
             print(pmid_clean)
             print("epub " + clean_epub)
             continue
           else:
             year_match = re.search(year_regex, current_text)
             if year_match:
               year_clean = year_match.group().split(" ")[0]
               paper_year_map[pmid_clean] = year_clean
               current_text = ""  
               print(pmid_clean)
               print("year " + year_clean)
               continue
             else:
               short_year_match = re.search(short_year_regex, current_text)
               if short_year_match:
                 clean_short_year = short_year_match.group().split(" ")[0]
                 paper_year_map[pmid_clean] = year_clean
                 current_text = ""   
                 print(pmid_clean)
                 print("short_year " + clean_short_year)
               else:
                 exception_year_match = re.search(short_year_regex, current_text)
                 if exception_year_match:
                   exception_short_year = exception_year_match.group().split(" ")[0]
                   current_text = ""
                   print(pmid_clean)
                   print("exception_year " + exception_short_year)
                   current_text = ""
         else:
          print("PMID NOT found")   
          current_text = ""
    return paper_year_map


def write_result(result_dict, file_name):
  """
  write extracted pmid and years in the file
  """ 
  with open(file_name, "w", encoding="utf8") as f:
    f.write(json.dumps(result_dict))    




results = "/content/drive/MyDrive/TextcorpusCreation/summary-geneexpres-set.txt"    
pmid_year_map = "/content/drive/MyDrive/TextcorpusCreation/pmid_year_map.json"  
pmid_year = create_paper_year_map(results)
write_result(pmid_year, pmid_year_map)
